In [26]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [27]:
!python -V

Python 3.11.8


In [28]:
import pickle
import pandas as pd

In [29]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [30]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [31]:
year = 2023
month = 3

In [32]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [33]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [34]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [39]:
df_result = df[['ride_id']].copy()

In [40]:
df_result

,ride_id
0,2023/03_0
1,2023/03_1
2,2023/03_2
3,2023/03_3
4,2023/03_4
...,...
3403761,2023/03_3403761
3403762,2023/03_3403762
3403763,2023/03_3403763
3403764,2023/03_3403764


In [41]:
df_result['predictions'] = y_pred

In [44]:
output_file = f'yellow_tripdata_{year:04d}-{month:02d}.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)